# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [60]:
csv_path = pd.read_csv("output_data/clean.csv")
csv_path

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Carnarvon,-24.8667,113.6333,72.36,48,0,19.69,AU,1666752973
1,Tuktoyaktuk,69.4541,-133.0374,12.20,92,75,13.80,CA,1666752973
2,Bilma,18.6853,12.9164,68.14,16,0,6.24,NE,1666752974
3,Hobart,-42.8794,147.3294,59.27,98,75,14.97,AU,1666752975
4,Jalpan,21.2333,-99.4833,66.00,66,91,2.35,MX,1666752975
...,...,...,...,...,...,...,...,...,...
583,Londonderry,54.9981,-7.3093,57.25,86,100,14.74,GB,1666753253
584,Geelong,-38.1471,144.3607,66.07,88,100,13.80,AU,1666752958
585,Kaabong,3.4836,34.1492,65.97,74,52,17.29,UG,1666753253
586,Mitsamiouli,-11.3847,43.2844,76.35,77,10,16.93,KM,1666753254


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [61]:
gmaps.configure(api_key = g_key)

In [62]:
locations = csv_path[['Lat','Lng']]
weight = csv_path["Humidity"]
heatmap_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100,
                                 point_radius=1)
fig = gmaps.figure()
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [63]:
ideal_weather_cond = csv_path.loc[(csv_path["Max Temp"] < 80) & (csv_path["Max Temp"] > 70) &
                                  (csv_path["Wind Speed"] < 10) & (csv_path["Cloudiness"] == 0)]
                                  
                                  
ideal_weather_cond =  ideal_weather_cond.dropna()                                 
ideal_weather_cond

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,Saint-Pierre,-21.3393,55.4781,73.90,64,0,5.75,RE,1666752992
213,Bānkura,23.2500,87.0667,78.89,57,0,3.29,IN,1666753070
253,Salalah,17.0151,54.0924,75.45,59,0,3.33,OM,1666753089
267,Caravelas,-17.7125,-39.2481,70.86,92,0,2.64,BR,1666753096
275,Pozo Colorado,-23.4923,-58.7972,78.66,47,0,9.33,PY,1666753100
353,Pāli,25.7667,73.3333,77.25,33,0,7.65,IN,1666753136
374,Ambilobe,-13.2000,49.0500,70.79,68,0,4.23,MG,1666752861
430,Asunción,-25.3007,-57.6359,74.64,69,0,4.61,PY,1666752914
438,Kattivākkam,13.2167,80.3167,78.89,58,0,6.76,IN,1666753176
489,Formosa,-26.1775,-58.1781,73.08,69,0,7.23,AR,1666753201


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
#store in hotel_df name
hotel_df = ideal_weather_cond

#add Hotel Name
hotel_df["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "keyword": "hotels",
    "type": "lodging",
    "key": g_key,
}

In [67]:
for index, row in hotel_df.iterrows():
    
    City = row["City"]
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    params["location"] = f"{Lat},{Lng}"
    
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f" The Closest {City} hotel is the {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
              
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")
              


 The Closest Saint-Pierre hotel is the Le Battant Des Lames.
 The Closest Bānkura hotel is the Hotel Saptarshi.
 The Closest Salalah hotel is the Salalah Gardens Hotel.
 The Closest Caravelas hotel is the Pousada Liberdade.
 The Closest Pozo Colorado hotel is the Hotel Pozo Colorado.
 The Closest Pāli hotel is the Hotel Minaxi Inn.
 The Closest Ambilobe hotel is the MOUNA HOTEL.
 The Closest Asunción hotel is the Crowne Plaza Asuncion, an IHG Hotel.
 The Closest Kattivākkam hotel is the THE CHIRAAG INN.
 The Closest Formosa hotel is the Los Ositos.
 The Closest Hamīrpur hotel is the Shyam Hotel.


In [68]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
39,Saint-Pierre,-21.3393,55.4781,73.90,64,0,5.75,RE,1666752992,Le Battant Des Lames
213,Bānkura,23.2500,87.0667,78.89,57,0,3.29,IN,1666753070,Hotel Saptarshi
253,Salalah,17.0151,54.0924,75.45,59,0,3.33,OM,1666753089,Salalah Gardens Hotel
267,Caravelas,-17.7125,-39.2481,70.86,92,0,2.64,BR,1666753096,Pousada Liberdade
275,Pozo Colorado,-23.4923,-58.7972,78.66,47,0,9.33,PY,1666753100,Hotel Pozo Colorado
353,Pāli,25.7667,73.3333,77.25,33,0,7.65,IN,1666753136,Hotel Minaxi Inn
374,Ambilobe,-13.2000,49.0500,70.79,68,0,4.23,MG,1666752861,MOUNA HOTEL
430,Asunción,-25.3007,-57.6359,74.64,69,0,4.61,PY,1666752914,"Crowne Plaza Asuncion, an IHG Hotel"
438,Kattivākkam,13.2167,80.3167,78.89,58,0,6.76,IN,1666753176,THE CHIRAAG INN
489,Formosa,-26.1775,-58.1781,73.08,69,0,7.23,AR,1666753201,Los Ositos


In [69]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [70]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))